In [5]:
import pandas as pd

# Set the file path of your source CSV file (tab-delimited) and output CSV file.
input_file = "transformed data 700 600 150/updated_flights_final.csv"
output_file = "transformed data 700 600 150/engine_runs_table.csv"

# Read the dataset with tab delimiter.
df = pd.read_csv(input_file, delimiter='\t')
#df.drop(['time_difference', 'First_Event_LessThan_5min_into_flight'], axis=1, inplace=True)

# Define the list of columns containing multiple values.
multi_value_cols = [
    'ENL_Engine_Run_Start_Times',
    'MOP_Engine_Run_Start_Times',
    'RPM_Engine_Run_Start_Times',
    'ENL_Engine_Run_Altitudes_MSL', 
    'MOP_Engine_Run_Altitudes_MSL',
    'RPM_Engine_Run_Altitudes_MSL',
    'ENL_Engine_Run_Altitudes_AGL',
    'MOP_Engine_Run_Altitudes_AGL',
    'RPM_Engine_Run_Altitudes_AGL',
    'ENL_Noise_Registration_Times',
    'MOP_Noise_Registration_Times',
    'RPM_Noise_Registration_Times',
    'ENL_Noise_Registration_Altitudes_AGL',
    'MOP_Noise_Registration_Altitudes_AGL',
    'RPM_Noise_Registration_Altitudes_AGL',
    'engine_run_agls',
    'altitude_rate_pre_event_window (ft/s)', 
    'distance_traveled_during_window (miles)',
    'is_circling_during_pre_event_window',
    'terrain_elev_at_engine_run (ft)',
    'terrain_label_at_engine_run',
    'Engine_Start_Above_1000ft',
    'engine_run_times (s)',
    'height_gain_loss (ft)',
    'avg_speed_while_engine_running (knots)',
    'min_speed_while_engine_running (knots)',
    'max_speed_while_engine_running (knots)',
    'All_Min_TAS_knots',
    'All_Max_TAS_knots',
    'All_Avg_TAS_knots',
    'All_Post_Run_Min_TAS_knots',
    'All_Post_Run_Max_TAS_knots',
    'All_Post_Run_Avg_TAS_knots',
    'Min_Speed_While_Engine_Running_knots',
    'Max_Speed_While_Engine_Running_knots',
    'Avg_Speed_While_Engine_Running_knots',
    'max_noise_ENL',
    'max_noise_MOP',
    'max_noise_RPM'				    
]

# The multi-value columns are split by a comma.
mv_delimiter = ','

# Function to safely split cell values into a list.
def split_cell(cell):
    if pd.isna(cell):
        return []
    if isinstance(cell, str):
        # Split by comma and strip any extra spaces.
        return [x.strip() for x in cell.split(mv_delimiter) if x.strip() != '']
    # If the cell is not a string, wrap it in a list.
    return [cell]

# Apply the split operation to each of the multi-value columns.
for col in multi_value_cols:
    if col in df.columns:
        df[col] = df[col].apply(split_cell)

# Create a list to collect new rows (each row representing one engine run).
new_rows = []

# Iterate over each original row and expand the multi-value columns.
for idx, row in df.iterrows():
    # Use the 'engine_run_times (s)' column as reference to determine the number of engine runs.
    run_list = row.get('engine_run_times (s)', [])
    num_runs = len(run_list) if run_list else 1
    
    for i in range(num_runs):
        new_row = row.copy()
        # For each multi-value column, set the cell to the i-th element if available.
        for col in multi_value_cols:
            if col in df.columns:
                values = row[col]
                new_row[col] = values[i] if i < len(values) else pd.NA
        new_rows.append(new_row)

# Create a new DataFrame containing one row per engine run.
engine_runs_df = pd.DataFrame(new_rows).reset_index(drop=True)

# Optionally, inspect the first few rows.
print(engine_runs_df.head())




   index Date (MM/DD/YYYY)                       File                  Gtype  \
0     13        08/18/2020  2020-08-18-CNI-20Y-01.IGC  ASW 27-18 E (ASG-29E)   
1     15        06/27/2024               46RGJUM1.IGC              VENTUS 3F   
2     24        06/29/2018  2018-06-29-NKL-10Q-01.IGC  ASW 27-18 E (ASG-29E)   
3     25        06/09/2018  2018-06-09-CNI-20E-01.IGC  ASW 27-18 E (ASG-29E)   
4     25        06/09/2018  2018-06-09-CNI-20E-01.IGC  ASW 27-18 E (ASG-29E)   

  Flight Time  Start Time  End Time Landing  Start_Alt (ft MSL)  \
0     3:37:01      164459    202200    HOME                 652   
1     3:42:29      193703    231932    HOME                3713   
2     3:18:46      183835    215721    HOME                 301   
3     2:39:52      193132    221124    HOME                1578   
4     2:39:52      193132    221124    HOME                1578   

  Max Altitude (ft MSL/ft AGL)  ...  All_Post_Run_Min_TAS_knots  \
0                   7208[5847]  ...              

In [6]:
def get_engine_run_start_time(row):
    # Check if ENL_Engine_Run_Start_Times is not null/empty.
    if pd.notna(row["ENL_Engine_Run_Start_Times"]) and str(row["ENL_Engine_Run_Start_Times"]).strip() != "":
        return row["ENL_Engine_Run_Start_Times"]
    # Otherwise, check the MOP column.
    elif pd.notna(row["MOP_Engine_Run_Start_Times"]) and str(row["MOP_Engine_Run_Start_Times"]).strip() != "":
        return row["MOP_Engine_Run_Start_Times"]
    # Otherwise, check the RPM column.
    elif pd.notna(row["RPM_Engine_Run_Start_Times"]) and str(row["RPM_Engine_Run_Start_Times"]).strip() != "":
        return row["RPM_Engine_Run_Start_Times"]
    else:
        return np.nan

# Apply the function to each row.
engine_runs_df["Engine_Run_Start_Times"] = engine_runs_df.apply(get_engine_run_start_time, axis=1)


# Save the new table to a CSV file.
engine_runs_df.to_csv(output_file, index=False)